In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop2.7.tgz
!tar -xvf spark-3.2.1-bin-hadoop2.7.tgz
!pip install -q findspark
!pip install pyspark

tar: spark-3.2.1-bin-hadoop2.7.tgz: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=642252cd97467d4a7aae54dbd571a99c9e6fd9ad1c6bf3c10287e57965ee79d4
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [ ]:
import findspark

In [ ]:
findspark.init()
from pyspark import SparkContext
sc = SparkContext()

In [ ]:
sc= SparkContext.getOrCreate()

In [ ]:
wordsList =  ["abc", "apple", "apple", "orange", "watermelon", "seed", "apple_seed", "history", "happy"]
wordsRDD = sc.parallelize(wordsList)
wordsRDD.getNumPartitions()

2

In [ ]:
print(wordsRDD.collect())

['abc', 'apple', 'apple', 'orange', 'watermelon', 'seed', 'apple_seed', 'history', 'happy']


In [ ]:
wordsRDD = sc.parallelize(wordsList, 4)
wordsRDD.getNumPartitions()

4

In [ ]:
print(wordsRDD.collect())

['abc', 'apple', 'apple', 'orange', 'watermelon', 'seed', 'apple_seed', 'history', 'happy']


In [ ]:
a = sc.parallelize(range(10),5)
a.glom().collect()

[[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]]

In [ ]:
a.repartition(2).glom().collect()

[[4, 5, 6, 7, 8, 9], [0, 1, 2, 3]]

In [ ]:
a = sc.parallelize(range(12),4)
a.glom().collect()

[[0, 1, 2], [3, 4, 5], [6, 7, 8], [9, 10, 11]]

In [ ]:
a.coalesce(2).glom().collect()

[[0, 1, 2, 3, 4, 5], [6, 7, 8, 9, 10, 11]]

In [ ]:
rdd1 = wordsRDD.map(lambda x: (x, x[0]))

In [ ]:
rdd2 = rdd1.map(lambda x: (x[1], x[0]))
rdd3 = rdd2.map(lambda x: (x[0], (x[1],1)))
rdd3.collect()

[('a', ('abc', 1)),
 ('a', ('apple', 1)),
 ('a', ('apple', 1)),
 ('o', ('orange', 1)),
 ('w', ('watermelon', 1)),
 ('s', ('seed', 1)),
 ('a', ('apple_seed', 1)),
 ('h', ('history', 1)),
 ('h', ('happy', 1))]

In [ ]:
rdd3.reduceByKey(lambda a,b: a[1]+b[1]).collect()

Py4JJavaError: ignored

In [ ]:
wordsPairs = wordsRDD.map(lambda x:x)
wordsPairs.collect()

['abc',
 'apple',
 'apple',
 'orange',
 'watermelon',
 'seed',
 'apple_seed',
 'history',
 'happy']

In [ ]:
wordGrouped = rdd2.groupByKey()
wordGrouped.collect()

[('s', <pyspark.resultiterable.ResultIterable at 0x7a1f691c1a80>),
 ('a', <pyspark.resultiterable.ResultIterable at 0x7a1f690c29b0>),
 ('w', <pyspark.resultiterable.ResultIterable at 0x7a1f690c1780>),
 ('h', <pyspark.resultiterable.ResultIterable at 0x7a1f690c0c40>),
 ('o', <pyspark.resultiterable.ResultIterable at 0x7a1f690c13c0>)]

In [ ]:
wordGrouped.mapValues(list).collect()

[('s', ['seed']),
 ('a', ['abc', 'apple', 'apple', 'apple_seed']),
 ('w', ['watermelon']),
 ('h', ['history', 'happy']),
 ('o', ['orange'])]

In [ ]:
rdd4 = wordGrouped.mapValues(list)

In [ ]:
rdd5 = rdd4.map(lambda x: (x[0], len(x[1])))
rdd5.collect()

[('s', 1), ('a', 4), ('w', 1), ('h', 2), ('o', 1)]

In [ ]:
rdd2prime = rdd2.map(lambda x: (x[0], x[1]))
rdd2prime.collect()

[('a', 'abc'),
 ('a', 'apple'),
 ('a', 'apple'),
 ('o', 'orange'),
 ('w', 'watermelon'),
 ('s', 'seed'),
 ('a', 'apple_seed'),
 ('h', 'history'),
 ('h', 'happy')]

In [ ]:
rdd2prime = rdd2.map(lambda x: (x[0], [x[1]]))
rdd2prime.collect()

[('a', ['abc']),
 ('a', ['apple']),
 ('a', ['apple']),
 ('o', ['orange']),
 ('w', ['watermelon']),
 ('s', ['seed']),
 ('a', ['apple_seed']),
 ('h', ['history']),
 ('h', ['happy'])]

In [ ]:
a = ["apple"]
b = ["orange"]
a+b

['apple', 'orange']

In [ ]:
rdd2prime.reduceByKey(lambda a,b: a+b).collect()

[('s', ['seed']),
 ('a', ['abc', 'apple', 'apple', 'apple_seed']),
 ('w', ['watermelon']),
 ('h', ['history', 'happy']),
 ('o', ['orange'])]